In [1]:
# NEED sklearn ver 1.0.2
import os
os.add_dll_directory("C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.2/bin") 
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import pandas_datareader as web
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from pandas.plotting import register_matplotlib_converters
from alpaca_trade_api.rest import REST, TimeFrame, TimeFrameUnit
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest_async import gather_with_concurrency, AsyncRest
from alpaca_trade_api.stream import Stream
from alpaca_trade_api.common import URL
import joblib
import yfinance as yf
import time 
import datetime
import sys
import requests as requests_cache

# check gpu availability
print(tf.config.list_physical_devices('GPU'))

[]


In [2]:
# depends on ur account
api_key_id = "YOUR_API_KEY"
api_secret = "YOUR_SECRET-KEY"
base_url = "https://paper-api.alpaca.markets"
feed = "iex"  # change to "sip" if you have a paid account

# fixed variables
time_steps = 5
symbol = ["AAPL", "AMD", "ASML", "JPM", "META", 
        "MSFT","NVDA", "PFE", "TSLA", "TSM"]
f_columns = ['open','high','low','volume','trade_count','vwap']

# initialize variables
# long and short list contain symbol and supposed quantity
long = []
short = []
longSymbol = []
shortSymbol = []
oldPosList = []
livePrice = [0] * len(symbol)
# in symbolRanking, the index refers to corresponding symbol
symbolPercent = [0] * len(symbol)
lastBalanceTime = float(0.0)

In [3]:
# connect to alpaca_trade_api
print("Connecting to alpaca_trade_api...")
rest = AsyncRest(key_id=api_key_id,
                 secret_key=api_secret)

api = tradeapi.REST(key_id=api_key_id,
                    secret_key=api_secret,
                    base_url=base_url)
print("connected")

Connecting to alpaca_trade_api...
connected


In [15]:
portfolio = api.get_portfolio_history("2022-07-06", "2022-07-23", timeframe="15Min").df

In [4]:
# load tensorflow model
model = []
print("loading model...")
for i in range(len(symbol)):
    model_fileName = symbol[i] + "_model"
    mod = tf.keras.models.load_model(model_fileName)
    model.append(mod)
    print("loaded model: ", model_fileName)
print("model loaded successfully")

loading model...
loaded model:  AAPL_model
loaded model:  AMD_model
loaded model:  ASML_model
loaded model:  JPM_model
loaded model:  META_model
loaded model:  MSFT_model
loaded model:  NVDA_model
loaded model:  PFE_model
loaded model:  TSLA_model
loaded model:  TSM_model
model loaded successfully


In [5]:
# load the robust transformer
# NEED sklearn ver 1.0.2
f_transformer = []
close_transformer = []
print("loading transformer...")
for i in range(len(symbol)):
    f_transformer_filename = "f_transformer_" + symbol[i] + ".save"
    close_transformer_filename = "close_transformer_" + symbol[i] + ".save"

    f_tran = joblib.load(f_transformer_filename)
    close_tran = joblib.load(close_transformer_filename) 

    f_transformer.append(f_tran)
    close_transformer.append(close_tran)
print("transformer loaded successfully")

loading transformer...
transformer loaded successfully


In [6]:
# Wait for market to open.
def awaitMarketOpen():
    isOpen = api.get_clock().is_open
    while(not isOpen):
        clock = api.get_clock()
        openingTime = clock.next_open.replace(
            tzinfo=datetime.timezone.utc).timestamp()
        currTime = clock.timestamp.replace(
            tzinfo=datetime.timezone.utc).timestamp()
        timeToOpen = int((openingTime - currTime) / 60)
        print(str(timeToOpen) + " minutes til market open.")
        time.sleep(60)
        isOpen = api.get_clock().is_open

In [ ]:
#await market open
print("Waiting for market to open...")
awaitMarketOpen()
print("Market opened.")

In [ ]:
# First, cancel any existing orders so they don't impact our buying power.
print("clearing existing orders")
api.cancel_all_orders()
print("orders have been cleared successfully")

clearing existing orders
orders have been cleared successfully


In [ ]:
# Figure out when the market will close so we can prepare to sell beforehand.
clock = api.get_clock()
closingTime = clock.next_close.replace(tzinfo=datetime.timezone.utc).timestamp()/60
currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()/60
timeToClose = closingTime - currTime

#in minutes
timeToClose

118.13676424697042

In [ ]:
print(currTime)
clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()/60

27617157.508796133


27617157.508796133

In [ ]:
#clear all positions if 5 min left, else trade every 15 min
if (timeToClose < 5):
    print ("market closing in 1 minute, clearing all positions...")
    api.close_all_positions()
    print("positions closed, waiting for market close")

In [26]:
def getLivePrice(symbol:str, session):
    while True:
        try:
            data = yf.download(tickers= symbol, period='10m', interval="1m", progress = False, session = session, threads = False, timeout =2)
            return data.Close[len(data) - 1]
        except:
            print("Error downloading, trying again...")
            time.sleep(1)

In [ ]:
yf.download(tickers= symbol, period='10m', interval="1m", progress = False, session = session, threads = False, timeout =2)

In [75]:
session = requests_cache.Session()
price = getLivePrice("AAPL", session)
session.close()
price 

142.9199981689453

In [71]:
# predict percentage change
def predictPercentChange():
    session = requests_cache.Session()
    for i in range(len(symbol)):
        # get and save live price
        livePrice[i] = getLivePrice(symbol[i], session)

        # get past data
        pastData = api.get_bars(symbol[i], TimeFrame(15, TimeFrameUnit.Minute), adjustment='all').df

        # scale past input data
        pastData.loc[:, f_columns] = f_transformer[i].transform(pastData[f_columns].to_numpy())
        pastData['close'] = close_transformer[i].transform(pastData[['close']])

        # numpy to array
        past_X = []
        past_X.append(pastData.iloc[len(pastData) - time_steps:])
        past_X = np.array(past_X)

        # predict using past data
        future_Y = model[i].predict(past_X)

        # inverse scale of predicted price
        future_Y= close_transformer[i].inverse_transform(future_Y)
        future_Y= future_Y.flatten()

        # calculate percent change
        # symbolPercent is the predicted percent change
        symbolPercent[i] = float(((future_Y - livePrice[i]) * 100) / livePrice[i])
        print(symbol[i] + " had predicted percent change: " + str(symbolPercent[i]))

        time.sleep(1.2)
    
    session.close()

In [12]:
def getPosition():
    # predict percentage change
    predictPercentChange()

    # put stock in long and short list and determine buying amount
    # reset long and short list
    long[:] = []
    short[:] = []
    totalPercentChange = sum(map(abs, symbolPercent))
    buyingPower = float(api.get_account().equity)
    for i in range(len (symbol)):
        # determine absolute amount of buying
        # times 0.95 to avoid unable to take pos
        amount = buyingPower * (abs(symbolPercent[i])/ totalPercentChange)
        amount = amount * 0.95

        # determine quantity of buying
        qty = int(amount / livePrice[i])
        # if predicted rise, buy long
        if symbolPercent[i] > 0:
            long.append ([symbol[i], qty])
            
        # else sell short as predicted fall
        else:
            short.append ([symbol[i], qty])

In [13]:
# getPosition()

In [14]:
# Submit an order if quantity is above 0. 
def submitOrder(stock:str, qty:int, side:str, respond  = []):
    if(qty > 0):
        try:
            api.submit_order(stock, qty, side, "market", "day")
            print("Market order of | " + str(qty) + " " +
            stock + " " + side + " | completed.")
            respond.append(True)
        except:
            print("Order of | " + str(qty) + " " + stock +
                    " " + side + " | did not go through.")
            respond.append(False)
    else:
        print("Quantity is 0, order of | " + str(qty) +
            " " + stock + " " + side + " | completed.")
        respond.append(True)

In [15]:
# submitOrder("AAPL", 123, "buy")

In [16]:
# api.close_position("AAPL")

In [17]:
short

[]

In [18]:
# return index pos of symbol in short list
def shortListIndex(symbol:str):
    # find index in long list
    for i in range(len(short)):
        
        # if it is a match return 
        if (short[i][0] == symbol):
            return i
    
    # if index cannot be found crash the program
    sys.exit()

In [19]:
# return index pos of symbol in long list
def longListIndex(symbol:str):
    # find index in long list
    for i in range(len(long)):
        
        # if it is a match return 
        if (long[i][0] == symbol):
            return i
    
    # if index cannot be found crash the program
    sys.exit()

In [40]:
# supposed all position exist 
def adjustPos():
    
    for oldPos in oldPosList:

        oldQty = abs(int(float(oldPos.qty)))

        # Position is now not in long list
        if(longSymbol.count(oldPos.symbol) == 0):

            # Position is now not in short list either.  Clear position.
            if (shortSymbol.count(oldPos.symbol) == 0):
                api.close_position(oldPos.symbol)

            # Position is now in short list        
            else: 

                # find wanted qty of particular stock in short list
                shortQty = short[shortListIndex(oldPos.symbol)][1]

                # position was in long but now in short. clear position and sell short
                if (oldPos.side == "long"):
                    api.close_position(oldPos.symbol)
                    time.sleep(3)
                    submitOrder(oldPos.symbol, shortQty, "sell")
                
                # position was in short and is now in short list
                else:

                    # old quantity is what what we want, pass for now
                    if (oldQty == shortQty):
                        pass

                    # need to adjust qty
                    else:
                        diff = oldQty - shortQty  
                        # too much short, buy some back
                        if (diff > 0):
                            submitOrder(oldPos.symbol, abs(diff), "buy")
                        else:
                            submitOrder(oldPos.symbol, abs(diff), "sell")

        # position is now in long list
        else:

            # find wanted qty of particular stock in long list
            longQty = long[longListIndex(oldPos.symbol)][1]

            # position changed from short to long, clear old position and buy long
            if (oldPos.side == "short"):
                api.close_position(oldPos.symbol)
                time.sleep(3)
                submitOrder(oldPos.symbol, longQty, "buy")
            
            # position was in long and is now in long list
            else:

                # old quantity is what what we want, pass for now
                if (oldQty == longQty):
                    pass
                
                 # need to adjust qty
                else:
                    diff = oldQty - longQty  
                    # too much long, sell some out
                    if (diff > 0):
                        submitOrder(oldPos.symbol, abs(diff), "sell")
                    else:
                        submitOrder(oldPos.symbol, abs(diff), "buy")               

In [21]:
# supposed no existing position WIP
def takePos():

    # buy in all long
    for i in range(len(long)):
        submitOrder(long[i][0], long[i][1], "buy")

    # sell out all short
    for i in range(len(short)):
        submitOrder(short[i][0], short[i][1], "sell")    

In [57]:
# rebalance position 
def rebalance():

    # clear all orders 
    api.cancel_all_orders()

    # get list of positions
    getPosition()

    # print symbol and qty of long position
    longSymbol[:] = []
    for i in range(len(long)):
        longSymbol.append(long[i][0])  
    print("We are taking a long position in: " + str(long))
    # print symbol and qty of short position
    shortSymbol[:] = []
    for i in range(len(short)):
        shortSymbol.append(short[i][0])
    print("We are taking a short position in: " + str(short))

    global oldPosList
    oldPosList.clear()
    oldPosList = api.list_positions()
    # adjust position if oldPosList is not empty (i.e. position already exist)
    if len(oldPosList) > 0:
        print("adjusting position...")
        adjustPos()
        print("adjustment completed")

    # else submit orders according to list directly
    else:
        print("taking new position...")
        takePos()
        print("position submitted")

In [58]:
rebalance()  

1/1 [==============================] - 0s 20ms/step
AAPL had predicted percent change: -0.0808078721165657
1/1 [==============================] - 0s 21ms/step
AMD had predicted percent change: -0.12739241123199463
1/1 [==============================] - 0s 28ms/step
ASML had predicted percent change: 0.40552935004234314
1/1 [==============================] - 0s 17ms/step
JPM had predicted percent change: -0.22829844057559967
1/1 [==============================] - 0s 32ms/step
META had predicted percent change: -0.6401351690292358
1/1 [==============================] - 0s 19ms/step
MSFT had predicted percent change: -0.44512879848480225
1/1 [==============================] - 0s 20ms/step
NVDA had predicted percent change: -0.5357868671417236
1/1 [==============================] - 0s 36ms/step
PFE had predicted percent change: -0.20363867282867432
1/1 [==============================] - 0s 22ms/step
TSLA had predicted percent change: -0.45998436212539673
1/1 [==============================

In [14]:
%%capture
api.close_all_positions()